![Marvel_Heros](marvel_heros.jpg)

## DataScienceChallenge 2
### _ Marvel Social Graph _

Les héros Marvel et les comics dans lequels ils apparaissent ont été enrregistrés dans une base de données en graph (Neo4J). Ce type de base de données permet de stocker et requèter efficacement les relations entre ces éléments (noeuds).

Le langage de programmation est appelé Cypher. Voici un exemple de requète:

```cypher
MATCH (:character)-[relation]->(comics)
RETURN * LIMIT 1
```



### 1 - Quel comics contiennent le plus de personnages ?

In [2]:
#Connexion à la bdd
from py2neo import Graph, authenticate, Relationship
graph = Graph()
authenticate("localhost:7474","neo4j", "admin")


In [5]:
graph.cypher.execute('''
MATCH (c:character)-[r]->(x:comics)
WITH DISTINCT(x.name) AS COMICS, COUNT(r) AS NB_APP
ORDER BY NB_APP DESC 
RETURN COMICS, NB_APP LIMIT 5
''')

   | COMICS               | NB_APP
---+----------------------+--------
 1 | COC 1                |    111
 2 | MARVEL MYSTERY COMIC |     92
 3 | IW 3                 |     91
 4 | IW 1                 |     90
 5 | H2 279               |     87